In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
from FINDER_API import FINDER_API
from py_utils.FINDER_test_utils import *
import os
import tqdm
import numpy as np


# Specify and load model

In [3]:
# model_path =  'best_models/tsp_2d/nrange_15_20_len_1011194/'
# model_path =  'best_models/tsp_2d/nrange_15_20_len_1011362/'
# model_path =  'best_models/tsp_2d/nrange_15_20_len_1009797/'
# model_path = 'best_models/tsp_2d/nrange_15_20_len_1011914/'

# insert function which automatically searches for best model with specified name
model_name = 'AGNN_default_len_10113392076640768'
model_name = 'AGNN_RNN_len_10117276006490983'
model_name = 'AGNN_no_helperfunction_len_10488634773857972'
model_name = 'AGNN_no_help_only_start_end_+_edge_prob_2021-09-01_08:31:56_len_100233'
model_path = f'saved_models/tsp_2d/nrange_20_20/{model_name}'

config_path = f'{model_path}/config.txt'
api = FINDER_API(config_path=config_path)

# get best checkpoint path
best_ckpt_file_path = get_best_ckpt(model_path)
# load checkpoint into Finder
api.load_model(ckpt_path=best_ckpt_file_path)

Error when loading key 'fix_start_node' from external config file!
Using default value 1 instead!
Gpu available: True
Built with cuda: True
model sucessfully restored from file


# Specify test sets

In [4]:
# data_dir = 'test_sets/synthetic_nrange_10_20_1000/'
# data_dir = 'test_sets/synthetic_nrange_15_20_1000/'
# data_dir = 'test_sets/synthetic_nrange_40_50_1000/'
# data_dir = 'valid_sets/synthetic_nrange_10_20_200/'
# data_dir = 'valid_sets/synthetic_nrange_15_20_200/'
# data_dir = 'data/test_sets/tsp_min-n=15_max-n=20_num-graph=1000_type=random/'
# data_dir_0 = 'data/test_sets/tsp_min-n=15_max-n=20_num-graph=1000_type=random/'
# data_dir_1 = 'data/test_sets/tsp_min-n=40_max-n=50_num-graph=1000_type=random/'
# data_dir_2 = 'data/test_sets/tsp_min-n=50_max-n=100_num-graph=1000_type=random/'
# data_dir_3 = 'data/test_sets/tsp_min-n=100_max-n=200_num-graph=1000_type=random/'
data_dir_valid = 'data/valid_sets/synthetic_nrange_20_20_100'
data_dir_0 = 'data/test_sets/synthetic_n_20_1000'
data_dir_1 = 'data/test_sets/synthetic_n_50_1000'
test_dirs = [data_dir_0, data_dir_1]

# Run tests

## Run standard tests

In [5]:
search_strategy = 'greedy'

In [6]:
# sanity check
lengths, solutions, sol_times = api.run_test(data_dir_valid, scale_factor=0.000001)
approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir_valid, lengths)
print(mean_approx_ratio)

Loading test graphs...


100%|██████████| 100/100 [00:07<00:00, 13.63it/s]

1.0023290615996827


In [7]:
mean_approx_ratios = []
for data_dir in test_dirs[1::]:
    # run test
    lengths, solutions, sol_times = api.run_test(data_dir, scale_factor=0.000001)
    approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir, lengths)
    mean_approx_ratios.append(mean_approx_ratio)
    print(mean_approx_ratio)
    # save test results
    save_approx_ratios(data_dir, approx_ratios, model_name, suffix=search_strategy)
    save_solutions(data_dir, solutions, model_name, suffix=search_strategy)
    save_lengths(data_dir, lengths, model_name, suffix=search_strategy)

Loading test graphs...


100%|██████████| 1000/1000 [05:30<00:00,  3.03it/s]


1.1686082125293842
Saving approximation ratios...


100%|██████████| 1000/1000 [00:08<00:00, 118.35it/s]


Saving solutions...


100%|██████████| 1000/1000 [00:08<00:00, 115.59it/s]


Saving solution lengths...


100%|██████████| 1000/1000 [00:08<00:00, 118.43it/s]


## Run tests with non-greedy search strategy


In [6]:
# use beam search during testing
search_strategy = 'beam_search+'
beam_width = 128
batch_size = 64

api.DQN.cfg['search_strategy'] = search_strategy
api.DQN.cfg['beam_width'] = beam_width
api.DQN.cfg['BATCH_SIZE'] = batch_size

suffix = f'{search_strategy}_{beam_width}'

In [7]:
# sanity check
lengths, solutions, sol_times = api.run_test(data_dir_valid, scale_factor=0.000001)
approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir_valid, lengths)
print(mean_approx_ratio)

Loading test graphs...


  0%|          | 0/100 [00:00<?, ?it/s]

[array([0.        , 0.05266373, 0.05233359, 0.05243742, 0.05360101,
       0.05379968, 0.05259506, 0.05248654, 0.05246247, 0.05253681,
       0.05223233, 0.05227016, 0.05269239, 0.05222086, 0.05258166,
       0.05255442, 0.05218754, 0.05297386, 0.0526193 , 0.05275117])]
[array([0.        , 0.05613613, 0.05495909, 0.0553721 , 0.05644114,
       0.        , 0.0560961 , 0.0551951 , 0.05521202, 0.05581897,
       0.05502964, 0.05488208, 0.05743409, 0.05486886, 0.0560315 ,
       0.05538028, 0.05481994, 0.05579386, 0.05537352, 0.0551556 ]), array([0.        , 0.05703852, 0.05509158, 0.05557   , 0.        ,
       0.05646338, 0.05596448, 0.05542312, 0.0554368 , 0.05588395,
       0.05513795, 0.0550261 , 0.05587095, 0.05493265, 0.0554326 ,
       0.05555922, 0.05488576, 0.05551943, 0.05558464, 0.05517887]), array([0.        , 0.05529643, 0.05537515, 0.05501632, 0.05579468,
       0.05598279, 0.05517007, 0.05512194, 0.05504598, 0.05514374,
       0.05554799, 0.05523639, 0.05567406, 0.05530065,

  0%|          | 0/100 [00:00<?, ?it/s]

[array([0.        , 0.05942912, 0.05815748, 0.05857887, 0.05973183,
       0.        , 0.05938604, 0.05839208, 0.0584087 , 0.05907383,
       0.05823613, 0.05807591, 0.060825  , 0.05807456, 0.05986736,
       0.05858006, 0.05803139, 0.        , 0.05857351, 0.05857814]), array([0.        , 0.06047178, 0.05830277, 0.05877976, 0.        ,
       0.06009403, 0.05926103, 0.05863018, 0.05864574, 0.05914586,
       0.05834169, 0.05821921, 0.        , 0.05811341, 0.05915885,
       0.05877836, 0.05809112, 0.05876192, 0.05880565, 0.05839864]), array([0.        , 0.05988525, 0.05818119, 0.05858725, 0.05977897,
       0.        , 0.        , 0.05839995, 0.05841061, 0.05954171,
       0.05824598, 0.05808039, 0.06084112, 0.05805816, 0.0593325 ,
       0.05859893, 0.05803309, 0.059049  , 0.05859262, 0.0583833 ]), array([0.        , 0.05795838, 0.06033255, 0.05840946, 0.05848006,
       0.        , 0.05789005, 0.05880809, 0.05866201, 0.0580637 ,
       0.        , 0.05925186, 0.05829958, 0.05924772, 

IndexError: list index out of range

In [ ]:
mean_approx_ratios = []
for data_dir in test_dirs[0:1]:
    # run test
    lengths, solutions, sol_times = api.run_test(data_dir, scale_factor=0.000001)
    approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir, lengths)
    mean_approx_ratios.append(mean_approx_ratio)
    print(mean_approx_ratio)
    # save test results
    save_approx_ratios(data_dir, approx_ratios, model_name, suffix=search_strategy)
    save_solutions(data_dir, solutions, model_name, suffix=search_strategy)
    save_lengths(data_dir, lengths, model_name, suffix=search_strategy)

# Test helper function

In [ ]:
graph = graph_list[0]
length, solution, time_ = dqn.Evaluate(g=graph)

In [ ]:
solution, length

In [ ]:
# check whether length calculation is correct
def calc_tour_length(graph, solution):
    tot_len = 0
    for i in range(np.array(solution).shape[0]):
        if i == np.array(solution).shape[0] - 1:
            tot_len += graph[solution[i]][solution[0]]['weight']
        else:
            tot_len += graph[solution[i]][solution[i + 1]]['weight']
    return tot_len

In [ ]:
def add_node(graph, cur_sol, new_node):
    cur_dist = 10000000.0
    for i in range(0, np.array(cur_sol).shape[0]):
        if i + 1 == np.array(cur_sol).shape[0]:
            adj = cur_sol[0]
        else:
            adj = cur_sol[i + 1]
        if cur_sol[i] == adj:
            cost = graph[new_node][cur_sol[i]]['weight'] + graph[new_node][adj]['weight']
        else:
            cost = graph[new_node][cur_sol[i]]['weight'] + graph[new_node][adj]['weight'] - graph[cur_sol[i]][adj]['weight']
        if cost < cur_dist:
            cur_dist = cost
            pos = i + 1
    cur_sol.insert(pos, new_node)
    return  cur_dist

In [ ]:
def calc_helper_tour(g):
    cur_sol = [0]
    rem_nodes = list(np.arange(g.number_of_nodes()))
    rem_nodes.remove(0)
    tour_length = 0
    while np.array(cur_sol).shape[0] < g.number_of_nodes():
        if np.array(rem_nodes).shape[0]-1 == 0:
            rand_idx = 0
        else:
            rand_idx = np.random.randint(0, np.array(rem_nodes).shape[0]-1)
        new_node = rem_nodes[rand_idx]
        tour_length += add_node(g, cur_sol, new_node)
        rem_nodes.remove(new_node)
    return tour_length, cur_sol


In [ ]:
max_result = -np.inf
min_result = np.inf
for i in range(100):
    help_lengths = []
    for graph in graph_list:
        help_lengths.append(calc_helper_tour(graph)[0])

    true_lengths = None
    with open(data_dir+'lengths.txt', 'r') as f:
        lines = f.readlines()
        lines = [float(line.split(':')[-1].strip()) for line in lines]
    true_lengths = lines
    result = np.mean([length[0]/length[1] for length in zip(help_lengths, true_lengths)])
    if result < min_result:
        min_result = result
    if result > max_result:
        max_result = result

In [ ]:
print(min_result)
print(max_result)